# Importing necessary libraries

In [0]:
import numpy
import pandas
from keras.models import Sequential, Model
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

# Making Preperations

In [31]:
from google.colab import files
uploaded = files.upload()

Saving iris.csv to iris (1).csv


In [0]:
# load dataset
dataframe = pandas.read_csv("iris.csv", header=None)
dataset = dataframe.values
X = dataset[:,0:4].astype(float)
Y = dataset[:,4]

In [0]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)


# Creating a basline model 

In [0]:
# define baseline model
def baseline_model():
	# create model
  model = Sequential()
  model.add(Dense(8, activation="relu", input_shape = (X.shape[1],)))
  model.add(Dense(3, activation="softmax"))

  # Compile model
  model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

  return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
#Creating the keras classifier
estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 97.33% (4.42%)


#Tuning Layers and Number of Neurons in The Model


# Creating a smaller network

In [0]:
# define smaller model
def smaller_model():
	# create model
  model = Sequential()
  model.add(Dense(4, activation="relu", input_shape = (X.shape[1],)))
  model.add(Dense(3, activation="softmax"))

  # Compile model
  model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

  return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
#Creating the keras classifier
estimator = KerasClassifier(build_fn=smaller_model, epochs=200, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Smaller: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Smaller: 82.67% (24.07%)


#Creating a larger network

In [0]:
# define larger model
def larger_model():
	# create model
  model = Sequential()
  model.add(Dense(16, activation="relu", input_shape = (X.shape[1],)))
  model.add(Dense(8, activation="relu"))
  model.add(Dense(3, activation="softmax"))

  # Compile model
  model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

  return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
#Creating the keras classifier
estimator = KerasClassifier(build_fn=larger_model, epochs=200, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Larger: 96.00% (5.33%)


# Creating a model that overfits

In [0]:
# define overfit model
def overfit_model():
	# create model
  model = Sequential()
  model.add(Dense(512, activation="relu", input_shape = (X.shape[1],)))
  model.add(Dense(128, activation="relu"))
  model.add(Dense(3, activation="softmax"))

  # Compile model
  model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

  return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
#Creating the keras classifier
estimator = KerasClassifier(build_fn=overfit_model, epochs=300, batch_size=10, verbose=0)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Overfits: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Overfits: 97.33% (4.42%)


# Optimized Model

In [34]:
# define overfit model
def optimized_model():
	# create model
  model = Sequential()
  model.add(Dense(10, activation="relu", input_shape = (X.shape[1],)))
  model.add(Dense(3, activation="softmax"))

  # Compile model
  model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

  return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
#Creating the keras classifier
estimator = KerasClassifier(build_fn=optimized_model, epochs=150, batch_size=7, verbose=0)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Optimized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Optimized: 98.00% (3.06%)


# Rewriting the code with Functional API

In [0]:
# define the fuctional API
def fucn_API():
  inputs = Input(shape= X.shape[1],)
  hidden = Dense(8, activation= "relu")(inputs)
  output = Dense(3, activation= "softmax")(hidden)
  model = Model(inputs= inputs, outputs = output)
  #compile model 
  model.compile(optmizer="adam",
               loss="categorical_crossentropy", 
               metrics = ["accuracy"])
  return model

# Rewriting the code with Model Sub classing

In [0]:
class MyModel(Model):
  def __init__(self):
     super(MyModel, self).__init__()
     # Define your layers here.
     inputs = Input(shape=(X.shape[1],))
     self.dense_1 = Dense(32, activation="relu")
     self.dense_2 = Dense(30, activation="softmax")
  def call(self, inputs):
     # Define your forward pass here,
     # using layers you previously defined in `__init__`
     x = self.dense_1(inputs)
     return self.dense_2(x)
def model_subclass():
  
  model = MyModel()
  model.compile(loss = "binary_crossentropy",
                optimizer = "rmsprop",
                metrics = ["accuracy"]
  )
  return model

# Model evaluation without using Scikit learn

In [0]:
import numpy as np

k = 10
num_val_samples = len(X) // k
num_epochs = 200
all_scores = []
for i in range(k):
    print('processing fold #', i)
    # Prepare the encoded_Y[i * num_val_samples: (i + 1) * num_val_samples]
    val_X = X[i * num_val_samples: (i + 1) * num_val_samples]
    val_Y = encoded_Y[i * num_val_samples: (i + 1) * num_val_samples]
    # Prepare the training data: data from all other partitions
    partial_X = np.concatenate(
        [X[:i * num_val_samples],
         X[(i + 1) * num_val_samples:]],
        axis=0)
    partial_Y = np.concatenate(
        [encoded_Y[:i * num_val_samples],
         encoded_Y[(i + 1) * num_val_samples:]],
        axis=0)

    # Build the Keras model (already compiled)
    model = optimized_model()
    
    model.compile(optimizer='adam',
    loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    # Train the model (in silent mode, verbose=0)
    model.fit(partial_X, partial_Y, epochs=num_epochs, batch_size=5, verbose=0)
    #          epochs=num_epochs,validation_split=10, shuffle=True, batch_size=5, verbose=0)
    # Evaluate the model on the validation data
    val_err, val_acc = model.evaluate(val_X, val_Y, verbose=0)
    all_scores.append(val_acc)

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3
processing fold # 4
processing fold # 5
processing fold # 6
processing fold # 7
processing fold # 8
processing fold # 9


In [36]:
print("Standardized: %.2f%% (%.2f%%)" % (np.mean(all_scores)*100, np.std(all_scores)*100))


Standardized: 94.67% (8.33%)
